In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
import re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import spacy
nlp = spacy.load('zh_core_web_lg')

zh_sw = spacy.lang.zh.stop_words.STOP_WORDS
punctuation = string.punctuation + '】' + '【' + '：' + '。' + '，' + '！' + '？'

In [2]:
df = pd.read_csv('microblogPCU/post.csv', encoding='GBK')

In [3]:
df.head()

,post_id,scratch_time,post_time,content,image,poster,poster_id,poster_url,repost_num,comment_num,repost_post_id
0,3.777580e+15,2014/11/17 10:30,2014/11/16 16:45,爆美来袭独家实拍质量看的见招微信代理一件代发专卖国际品牌名鞋品质赢得市场诚信赚得人心V信：a...,http://tp1.sinaimg.cn/2364770064/50/22844411852/1,新百伦商城,2364770064,http://weibo.com/527loveyou,0,0,NaN
1,3.777620e+15,2014/11/17 15:45,2014/11/16 19:58,【奥汀羽酵素反馈】效果不好也不会卖这么好。正常吃饭，不用节食，不用运动，健康绿色没有副作用。...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN
2,3.777620e+15,2014/11/17 20:08,2014/11/16 19:58,我有话说【世界最穷总统拒绝酋长高价收购其老旧座驾】共产党员？http://t.cn/RzvUwBM,http://tp2.sinaimg.cn/2747799853/50/0/1,IT经济学,2747799853,http://weibo.com/u/2747799853,0,0,NaN
3,3.777620e+15,2014/11/17 20:08,2014/11/16 19:57,【蚕丝皂】吸附黑头的能力一级棒，清洁不选它是你没眼光[撇嘴][撇嘴]西游记申请吉尼斯百元哥被...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN
4,3.777620e+15,2014/11/17 20:09,2014/11/16 19:57,我活得很好！//【世界最穷总统拒绝酋长高价收购其老旧座驾】http://t.cn/RzvUwDk,http://tp2.sinaimg.cn/5136475477/50/0/1,很久很久58172,5136475477,http://weibo.com/u/5136475477,0,0,NaN


I'm going to start small here and just work with the top 5 rows.

In [4]:
df_small = df.head()

In [5]:
df_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   post_id         5 non-null      float64
 1   scratch_time    5 non-null      object 
 2   post_time       5 non-null      object 
 3   content         5 non-null      object 
 4   image           5 non-null      object 
 5   poster          5 non-null      object 
 6   poster_id       5 non-null      int64  
 7   poster_url      5 non-null      object 
 8   repost_num      5 non-null      int64  
 9   comment_num     5 non-null      int64  
 10  repost_post_id  0 non-null      float64
dtypes: float64(2), int64(3), object(6)
memory usage: 568.0+ bytes


In [6]:
def cleaner(text):
    text = re.sub(r'[%s]' % re.escape(punctuation), '', text)
    text = re.sub(r'http\S+', '', text)
    text = nlp(text)
    tokens = [token for token in text if not token.is_stop]
    # turn tokens back into strings (unable to rejoin text when they were spacy.token objects)
    strings = [str(token) for token in tokens]
    # now join
    text = ' '.join(strings)
    
    return text

In [7]:
df_small['clean_text'] = df_small.content.apply(lambda x: cleaner(x))

<ipython-input-7-f6acd5478c07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['clean_text'] = df_small.content.apply(lambda x: cleaner(x))


In [8]:
df_small

,post_id,scratch_time,post_time,content,image,poster,poster_id,poster_url,repost_num,comment_num,repost_post_id,clean_text
0,3.777580e+15,2014/11/17 10:30,2014/11/16 16:45,爆美来袭独家实拍质量看的见招微信代理一件代发专卖国际品牌名鞋品质赢得市场诚信赚得人心V信：a...,http://tp1.sinaimg.cn/2364770064/50/22844411852/1,新百伦商城,2364770064,http://weibo.com/527loveyou,0,0,NaN,爆美 来袭 独家 实拍 质量 招 微信 代理 件 代发 专卖 国际 品牌 名鞋 品质 赢得 ...
1,3.777620e+15,2014/11/17 15:45,2014/11/16 19:58,【奥汀羽酵素反馈】效果不好也不会卖这么好。正常吃饭，不用节食，不用运动，健康绿色没有副作用。...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,奥汀羽 酵素 反馈 效果 卖 吃饭 不用 节食 不用 运动 健康 绿色 副作用 奥汀羽 棒 ...
2,3.777620e+15,2014/11/17 20:08,2014/11/16 19:58,我有话说【世界最穷总统拒绝酋长高价收购其老旧座驾】共产党员？http://t.cn/RzvUwBM,http://tp2.sinaimg.cn/2747799853/50/0/1,IT经济学,2747799853,http://weibo.com/u/2747799853,0,0,NaN,世界 穷 总统 拒绝 酋长 高价 收购 老旧 座驾 共产党员
3,3.777620e+15,2014/11/17 20:08,2014/11/16 19:57,【蚕丝皂】吸附黑头的能力一级棒，清洁不选它是你没眼光[撇嘴][撇嘴]西游记申请吉尼斯百元哥被...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,蚕 丝皂 吸附 黑头 能力 一级 棒 清洁 选 眼光 撇嘴 撇嘴西 游记 申请 吉尼斯百 元...
4,3.777620e+15,2014/11/17 20:09,2014/11/16 19:57,我活得很好！//【世界最穷总统拒绝酋长高价收购其老旧座驾】http://t.cn/RzvUwDk,http://tp2.sinaimg.cn/5136475477/50/0/1,很久很久58172,5136475477,http://weibo.com/u/5136475477,0,0,NaN,世界 穷 总统 拒绝 酋长 高价 收购 老旧 座驾


Ok so now I think we're ready to start topic modeling!

In [9]:
vectorizer = TfidfVectorizer()

In [10]:
X = vectorizer.fit_transform(df_small.clean_text)

In [11]:
words = np.array(vectorizer.get_feature_names())

In [12]:
X

<5x81 sparse matrix of type '<class 'numpy.float64'>'
	with 111 stored elements in Compressed Sparse Row format>

In [13]:
vectorizer.get_feature_names()

['1835832094',
 '2015',
 'qq',
 'v信asicshoes',
 '一级',
 '不用',
 '专卖',
 '世界',
 '丝皂',
 '中国',
 '京东',
 '代发',
 '代理',
 '健康',
 '元哥',
 '共产党员',
 '副作用',
 '南航',
 '去世',
 '反馈',
 '吃饭',
 '吉尼斯百',
 '名鞋',
 '吸附',
 '品牌',
 '品质',
 '商城',
 '国际',
 '奥汀羽',
 '妈妈',
 '实拍',
 '市场',
 '广州',
 '座驾',
 '强北',
 '得人心',
 '微信',
 '总统',
 '拒绝',
 '指甲',
 '捡手机',
 '撇嘴',
 '撇嘴西',
 '收购',
 '效果',
 '日历',
 '春运',
 '李夏怡',
 '李大霄',
 '李秀贤',
 '来袭',
 '求约',
 '油价',
 '清洁',
 '游记',
 '爆仓',
 '爆美',
 '独家',
 '申请',
 '百元哥',
 '皮肤长',
 '眼光',
 '结婚',
 '绿色',
 '老旧',
 '能力',
 '节食',
 '苏宁华',
 '西游记',
 '诚信',
 '质量',
 '购票',
 '赢得',
 '车展',
 '运动',
 '酋长',
 '酵素',
 '金三顺',
 '阿里巴巴iphone5',
 '高价',
 '黑头']

In [14]:
nmf = NMF(n_components=2, random_state=1, solver='mu')
W = nmf.fit_transform(X)
H = nmf.components_

/Users/josephtanner/opt/anaconda3/envs/chinese_research/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [15]:
for i, topic in enumerate(H):
    print('Topic {}: {}'.format(i, ', '.join([str(x) for x in words[topic.argsort()[-5:]]])))

Topic 0: 拒绝, 高价, 收购, 座驾, 老旧
Topic 1: 皮肤长, 申请, 妈妈, 不用, 奥汀羽


In [16]:
def topic_table(model, feature_names, n_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        t = (topic_idx)
        topics[t] = [feature_names[i] for i in top_words(topic, n_top_words)]
    
    return pd.DataFrame(topics)

In [17]:
def top_words(topic, n_top_words):
    return topic.argsort()[:-n_top_words -1:-1]

In [18]:
def whitespace_tokenizer(text):
    pattern = r'(?u)\b\w\w+\b'
    tokenizer_regex = RegexpTokenizer(pattern)
    tokens = tokenizer_regex.tokenize(text)
    
    return tokens

In [19]:
def unique_words(text):
    ulist = []
    [ulist.append(x) for x in text if x not in ulist]
    
    return ulist

In [20]:
docweights = nmf.transform(vectorizer.transform(df_small.clean_text))

In [21]:
n_top_words = 8

topic_df = topic_table(nmf, words, n_top_words).T

topic_df['topics'] = topic_df.apply(lambda x: [' '.join(x)], axis = 1) # join each word into a list
topic_df['topics'] = topic_df['topics'].str[0] # remove list brackets
topic_df['topics'] = topic_df['topics'].apply(lambda x: whitespace_tokenizer(x)) # tokenize
topic_df['topics'] = topic_df['topics'].apply(lambda x: unique_words(x)) # remove duplicate words
topic_df['topics'] = topic_df['topics'].apply(lambda x: [' '.join(x)]) # rejoin into list
topic_df['topics'] = topic_df['topics'].str[0] # remove list brackets

In [22]:
topic_df

,0,1,2,3,4,5,6,7,topics
0,老旧,座驾,收购,高价,拒绝,酋长,世界,总统,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
1,奥汀羽,不用,妈妈,申请,皮肤长,结婚,元哥,李秀贤,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤


In [23]:
topic_df = topic_df['topics'].reset_index()

In [24]:
topic_df.columns = ['topic_num', 'topics']

In [25]:
topic_df

,topic_num,topics
0,0,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
1,1,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤


In [26]:
post_id = df_small['post_id'].tolist()

In [27]:
df_temp = pd.DataFrame({'post_id': post_id,
                      'topic_num': docweights.argmax(axis=1)})

In [28]:
merged_topic = df_temp.merge(topic_df, on='topic_num', how='left')

In [29]:
df_topics = pd.merge(df_small, merged_topic, on='post_id', how='left')

In [30]:
df_topics

,post_id,scratch_time,post_time,content,image,poster,poster_id,poster_url,repost_num,comment_num,repost_post_id,clean_text,topic_num,topics
0,3.777580e+15,2014/11/17 10:30,2014/11/16 16:45,爆美来袭独家实拍质量看的见招微信代理一件代发专卖国际品牌名鞋品质赢得市场诚信赚得人心V信：a...,http://tp1.sinaimg.cn/2364770064/50/22844411852/1,新百伦商城,2364770064,http://weibo.com/527loveyou,0,0,NaN,爆美 来袭 独家 实拍 质量 招 微信 代理 件 代发 专卖 国际 品牌 名鞋 品质 赢得 ...,0,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
1,3.777620e+15,2014/11/17 15:45,2014/11/16 19:58,【奥汀羽酵素反馈】效果不好也不会卖这么好。正常吃饭，不用节食，不用运动，健康绿色没有副作用。...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,奥汀羽 酵素 反馈 效果 卖 吃饭 不用 节食 不用 运动 健康 绿色 副作用 奥汀羽 棒 ...,1,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤
2,3.777620e+15,2014/11/17 15:45,2014/11/16 19:58,【奥汀羽酵素反馈】效果不好也不会卖这么好。正常吃饭，不用节食，不用运动，健康绿色没有副作用。...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,奥汀羽 酵素 反馈 效果 卖 吃饭 不用 节食 不用 运动 健康 绿色 副作用 奥汀羽 棒 ...,0,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
3,3.777620e+15,2014/11/17 15:45,2014/11/16 19:58,【奥汀羽酵素反馈】效果不好也不会卖这么好。正常吃饭，不用节食，不用运动，健康绿色没有副作用。...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,奥汀羽 酵素 反馈 效果 卖 吃饭 不用 节食 不用 运动 健康 绿色 副作用 奥汀羽 棒 ...,1,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤
4,3.777620e+15,2014/11/17 15:45,2014/11/16 19:58,【奥汀羽酵素反馈】效果不好也不会卖这么好。正常吃饭，不用节食，不用运动，健康绿色没有副作用。...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,奥汀羽 酵素 反馈 效果 卖 吃饭 不用 节食 不用 运动 健康 绿色 副作用 奥汀羽 棒 ...,0,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
5,3.777620e+15,2014/11/17 20:08,2014/11/16 19:58,我有话说【世界最穷总统拒绝酋长高价收购其老旧座驾】共产党员？http://t.cn/RzvUwBM,http://tp2.sinaimg.cn/2747799853/50/0/1,IT经济学,2747799853,http://weibo.com/u/2747799853,0,0,NaN,世界 穷 总统 拒绝 酋长 高价 收购 老旧 座驾 共产党员,1,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤
6,3.777620e+15,2014/11/17 20:08,2014/11/16 19:58,我有话说【世界最穷总统拒绝酋长高价收购其老旧座驾】共产党员？http://t.cn/RzvUwBM,http://tp2.sinaimg.cn/2747799853/50/0/1,IT经济学,2747799853,http://weibo.com/u/2747799853,0,0,NaN,世界 穷 总统 拒绝 酋长 高价 收购 老旧 座驾 共产党员,0,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
7,3.777620e+15,2014/11/17 20:08,2014/11/16 19:58,我有话说【世界最穷总统拒绝酋长高价收购其老旧座驾】共产党员？http://t.cn/RzvUwBM,http://tp2.sinaimg.cn/2747799853/50/0/1,IT经济学,2747799853,http://weibo.com/u/2747799853,0,0,NaN,世界 穷 总统 拒绝 酋长 高价 收购 老旧 座驾 共产党员,1,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤
8,3.777620e+15,2014/11/17 20:08,2014/11/16 19:58,我有话说【世界最穷总统拒绝酋长高价收购其老旧座驾】共产党员？http://t.cn/RzvUwBM,http://tp2.sinaimg.cn/2747799853/50/0/1,IT经济学,2747799853,http://weibo.com/u/2747799853,0,0,NaN,世界 穷 总统 拒绝 酋长 高价 收购 老旧 座驾 共产党员,0,老旧 座驾 收购 高价 拒绝 酋长 世界 总统
9,3.777620e+15,2014/11/17 20:08,2014/11/16 19:57,【蚕丝皂】吸附黑头的能力一级棒，清洁不选它是你没眼光[撇嘴][撇嘴]西游记申请吉尼斯百元哥被...,http://tp3.sinaimg.cn/2346769694/50/5704078058/0,孚禾静静_,2346769694,http://weibo.com/206445959,0,0,NaN,蚕 丝皂 吸附 黑头 能力 一级 棒 清洁 选 眼光 撇嘴 撇嘴西 游记 申请 吉尼斯百 元...,1,奥汀羽 不用 妈妈 申请 皮肤长 结婚 元哥 李秀贤


In [31]:
df_topics.post_id.value_counts()

3.777620e+15    16
3.777580e+15     1
Name: post_id, dtype: int64